In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_columns', None)

URL = 'https://www.pro-football-reference.com/teams/{team}/2021-snap-counts.htm'

team_arr = ['crd', 'atl', 'rav', 'buf', 'car', 'chi', 'cin', 'cle', 'dal', 'den', 'det', 'gnb', 'htx', 'clt', 'jax',\
    'kan', 'rai', 'sdg', 'ram', 'mia', 'min', 'nwe', 'nor', 'nyg', 'nyj', 'phi', 'pit', 'sfo', 'sea', 'tam', 'oti', 'was' ]

season_df = pd.DataFrame()


for team in team_arr:
    res = requests.get(URL.format(team=team))
    soup = bs(res.content, 'html.parser')
    table = soup.find('table', {'id': 'snap_counts'})
    df = pd.read_html(str(table))[0]
    df.columns = df.columns.droplevel(level=0)

    df = df.iloc[:, :4]
    off_skill_pos = ['QB', 'RB', 'WR', 'TE', 'FB']
    df= df.loc[df['Pos'].isin(off_skill_pos)]
    df['Pos'].mask(df['Pos'] == 'FB', 'RB', inplace=True)
    df = df.rename(columns={
    'Player':'Name',
    'Num': 'Snaps',
    'Pct': 'Snap%'
    })
    df['Team'] = team
    df = df.reindex(columns=[
    'Pos','Team','Name','Snaps','Snap%'
    ])

    df['Snap%'] = df['Snap%'].str.rstrip('%')
    df['Snap%'] = pd.to_numeric(df['Snap%'], errors='coerce')


    team_snap_total = ((df['Snaps'][0].astype('float')*100)//df['Snap%'][0])
    df['TeamSnaps'] = team_snap_total

    season_df = pd.concat([season_df,df])






In [113]:
season_df.to_csv('/Users/brandynklee/Python_Projects/fantasy_footballl_22/data/snapcount_2021.csv', index=False)